<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/005_Pipelines_Models_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🤖 Hugging Face Pipelines: What They Do

A **pipeline** in Hugging Face is a pre-wired wrapper that:
1. Tokenizes your input
2. Runs inference on a model
3. Decodes the output into readable text

Each pipeline is matched to a **task type** — and you choose the pipeline **based on what you want the model to do**.

---

## 📚 Summary Table: Pipelines, Tasks, and Models

| **Pipeline** | **What It Does** | **Example Use** | **Good Models** |
|--------------|------------------|------------------|------------------|
| `text-generation` | Predicts next tokens in freeform text | Autocomplete, writing assistants | `gpt2`, `falcon-rw-1b`, `DialoGPT` |
| `text2text-generation` | Translates one text into another (task-following) | Instructions → output, translation, summarization | `flan-t5-base`, `t5-small`, `bart-base` |
| `text-classification` | Labels input text with a category | Sentiment, spam detection | `bert-base-uncased`, `distilbert-base-uncased` |
| `question-answering` | Answers a question given a context | "What is his name?" → from a paragraph | `bert-large-uncased-whole-word-masking-finetuned-squad` |
| `summarization` | Condenses input text to a shorter version | News summaries, TL;DR | `bart-large-cnn`, `t5-base` |
| `translation` | Converts text from one language to another | English → French | `t5`, `mbart-large-50`, `opus-mt-en-fr` |
| `conversational` | Handles back-and-forth dialogue with memory | Chatbots with context | `DialoGPT`, `Blenderbot` |

---

## 🔍 Quick Cheat Sheet

### 🧠 `text-generation`
- **Goal**: Complete a sentence, story, or thought
- **Think**: "Once upon a time..."
- **Style**: Open-ended, creative
- **Model style**: GPT-style
- ✅ Use when you don’t need strict structure

---

### 🧠 `text2text-generation`
- **Goal**: Follow instructions and output something specific
- **Think**: “Summarize this article” or “Translate this to French”
- **Style**: Precise, controllable
- **Model style**: T5-style (encoder-decoder)
- ✅ Best for AI agents, tools, workflows

---

### 🧠 `text-classification`
- **Goal**: Predict a label (positive/negative, topic, intent)
- **Style**: Single answer from fixed set
- ✅ Use when the output should be one of a few choices

---

### 🧠 `question-answering`
- **Goal**: Find answer **in a given paragraph**
- **Style**: Not generative — it **extracts** answers from provided context
- ✅ Use in retrieval-augmented generation or open-book QA

---

### 🧠 `summarization`
- **Goal**: Shorten text while preserving meaning
- ✅ Use for reports, articles, transcripts

---

### 🧠 `translation`
- **Goal**: Translate between languages
- ✅ Specialized for multilingual support

---

### 🧠 `conversational`
- **Goal**: Maintain turn-by-turn memory across chats
- ✅ Use when you need memory in an assistant or chatbot

---

## 🤖 Which Pipeline for Agents?

| Agent Behavior | Best Pipeline | Why |
|----------------|---------------|-----|
| Follow instructions (tool use, info extraction) | `text2text-generation` | Clear prompt + clean output |
| Freeform conversation or idea expansion | `text-generation` | Open-ended creativity |
| Routing or intent detection | `text-classification` | You get structured labels |
| Summarize large blocks of text | `summarization` | Less creative, more compression |
| FAQ-style with a document | `question-answering` | Great with context chunks |



In [1]:
!pip install -q transformers huggingface_hub

## 🔮 `text-generation` Pipeline (GPT-style models)

### ✅ What It Does:
This type of model is trained to **predict the next word** given what came before. It’s not trying to follow instructions — it’s just continuing the flow of text.

Think of it as:
> 🧠 "Complete this sentence like a human would."

Each of these will return something reasonable — but maybe a little unpredictable or unstructured.





In [4]:
# supress warning
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)
from transformers import pipeline

# Load a small text-generation model
generator = pipeline("text-generation", model="gpt2")

# 1. Autocomplete a thought
prompt = "The future of AI agents is"
output = generator(prompt, max_new_tokens=30)[0]["generated_text"]
print("1️⃣ Completion:\n", output)

# 2. Start a story
prompt = "Once upon a time in a world powered by AI,"
output = generator(prompt, max_new_tokens=30)[0]["generated_text"]
print("2️⃣ Story:\n", output)

# 3. Give a list of ideas
prompt = "Three reasons to use AI agents are:"
output = generator(prompt, max_new_tokens=30)[0]["generated_text"]
print("3️⃣ List:\n", output)

1️⃣ Completion:
 The future of AI agents is murky, but a very real possibility," said Lawrence Berkeley National Laboratory professor and co-author Richard Moore, who holds a Ph.D. in computational
2️⃣ Story:
 Once upon a time in a world powered by AI, it's difficult to avoid. And if they can't catch us in sight through AI, they are too.

When you make a mistake,
3️⃣ List:
 Three reasons to use AI agents are:

Autoworking and AI agents must be configured based on the algorithm and are very efficient in the short-term.

Autow


### 🔥 Now Let's Test Its Limits

### 😬 Where `text-generation` *Fails*

#### ❌ 1. Follow Instructions (It won't!)

```python
prompt = "Translate this sentence into French: I love machine learning."
output = generator(prompt, max_new_tokens=30)[0]["generated_text"]
print(output)
```

→ You’ll probably get something like:

> Translate this sentence into French: I love machine learning. It’s a fascinating field that…

🙅 It **does not translate** — it continues the thought.

---

#### ❌ 2. Extract Information or Answer Questions Cleanly

```python
prompt = "What is the capital of France?"
output = generator(prompt, max_new_tokens=30)[0]["generated_text"]
print(output)
```

→ It might say something **correct**, but it could also say:

> What is the capital of France? The capital of France has long been known for its...

🙅 It wanders. No guarantee of a clean answer like `"Paris"`.

---

## 🧠 Why This Happens

- GPT-style models are **trained to generate flowing language**, not follow specific instructions.
- There's no native structure to the response.
- It doesn’t “understand” your prompt as a task.

---

## 🎯 Where `text2text-generation` Shines

Now you’ll see:
- Instruction-following
- Clean format
- Task-specific outputs

For example:

| Task | GPT2 (text-gen) | FLAN-T5 (text2text-gen) |
|------|------------------|--------------------------|
| Translate to French | Wanders off | `"J'aime l'apprentissage automatique"` |
| Answer a direct question | Talks in circles | `"Paris"` |
| Rephrase this sentence | Just continues | Rephrases as asked |


In [7]:
# Example 1: Translation attempt
prompt1 = "Translate this sentence into French: I love machine learning."
output1 = generator(prompt1, max_new_tokens=30)[0]["generated_text"]

# Example 2: Question answering attempt
prompt2 = "What is the capital of France?"
output2 = generator(prompt2, max_new_tokens=30)[0]["generated_text"]

# Pretty print
print("❌ Example 1: Translation with text-generation")
print("📝 Prompt:\n", prompt1)
print("🤖 Output:\n", output1)
print("\n" + "="*60 + "\n")

print("❌ Example 2: Question answering with text-generation")
print("📝 Prompt:\n", prompt2)
print("🤖 Output:\n", output2)
print("\n" + "="*60 + "\n")


❌ Example 1: Translation with text-generation
📝 Prompt:
 Translate this sentence into French: I love machine learning.
🤖 Output:
 Translate this sentence into French: I love machine learning.

I love machine learning.

Because Machine Learning is a powerful tool, it's always worth looking at those examples of it in a broader


❌ Example 2: Question answering with text-generation
📝 Prompt:
 What is the capital of France?
🤖 Output:
 What is the capital of France?

Let me say that I've seen that capital in France from the French side, where they were a real hard-working country, not a





## 📚 Understanding `text-generation` vs. `text2text-generation`

### 🔮 `text-generation`
- **Model Type**: GPT-style (causal language models)
- **Pipeline**: `text-generation`
- **Purpose**: Freeform text continuation
- **Strengths**:
  - Great for creative writing, storytelling, or idea expansion
  - Can generate flowing, human-like text
- **Limitations**:
  - Does *not* follow instructions reliably
  - Unstructured output — hard to parse or control
  - Poor at task-specific prompts like “translate this” or “summarize this”

> **Examples of good use**:  
> `"Once upon a time..."` → continues the story  
> `"Write a fantasy scene about dragons..."`

---

### 🧠 `text2text-generation`
- **Model Type**: T5, FLAN, BART (encoder-decoder)
- **Pipeline**: `text2text-generation`
- **Purpose**: Instruction following — turns input text into output text
- **Strengths**:
  - Designed for structured tasks like translation, summarization, rephrasing, Q&A
  - More reliable and precise than `text-generation`
  - Better suited for use in AI agents and task automation
- **Limitations**:
  - Not great for longform creative writing
  - Tends to give short, controlled responses (not ideal for open-ended generation)

> **Examples of good use**:  
> `"Translate this into French: I love AI"`  
> `"Summarize this paragraph..."`  
> `"What is the capital of France?"` → `"Paris"`

---

### 🧪 Summary
| Feature                 | `text-generation`        | `text2text-generation`        |
|------------------------|--------------------------|-------------------------------|
| Style                  | Open-ended               | Instruction-following         |
| Structure              | Loose, unpredictable     | Clean and controlled           |
| Best for               | Storytelling, creative    | Translation, summarization     |
| Output Format          | Natural continuation     | Task-specific response         |
| Example Model          | `gpt2`                   | `flan-t5-base`, `t5-small`     |
| Common Mistake         | Tries to “think aloud”   | Tries to be brief and literal  |



In [9]:
# Load an instruction-following model
generator = pipeline("text2text-generation", model="google/flan-t5-base")

# --- Successful Prompts ---
prompts = [
    ("Translate this sentence into French: I love machine learning.", "🌍 Translation"),
    ("What is the capital of France?", "📍 Direct Question Answering"),
    ("Summarize: Artificial intelligence is a branch of computer science focused on creating intelligent machines that can perform tasks typically requiring human intelligence.", "🧾 Summarization"),
    ("Rephrase: I am not sure if I can help you.", "🗣️ Rephrasing")
]

print("✅ Successful Examples Using `text2text-generation`\n" + "="*60 + "\n")

for prompt, label in prompts:
    output = generator(prompt, max_new_tokens=60)[0]["generated_text"]
    print(f"{label}\n📝 Prompt:\n{prompt}\n🤖 Output:\n{output}\n" + "="*60 + "\n")


✅ Successful Examples Using `text2text-generation`

🌍 Translation
📝 Prompt:
Translate this sentence into French: I love machine learning.
🤖 Output:
J'aime l'apprentissage de la machine.

📍 Direct Question Answering
📝 Prompt:
What is the capital of France?
🤖 Output:
london

🧾 Summarization
📝 Prompt:
Summarize: Artificial intelligence is a branch of computer science focused on creating intelligent machines that can perform tasks typically requiring human intelligence.
🤖 Output:
Understand the science behind artificial intelligence.

🗣️ Rephrasing
📝 Prompt:
Rephrase: I am not sure if I can help you.
🤖 Output:
I am not sure if I can help you.



In [10]:
# --- Prompts text2text may struggle with ---
fail_prompts = [
    ("Continue the story: Once upon a time in a world of intelligent machines,", "📖 Freeform Storytelling"),
    ("Write a poem about space exploration.", "🧑‍🚀 Creative Writing")
]

print("❌ Limitations of `text2text-generation`\n" + "="*60 + "\n")

for prompt, label in fail_prompts:
    output = generator(prompt, max_new_tokens=60)[0]["generated_text"]
    print(f"{label}\n📝 Prompt:\n{prompt}\n🤖 Output:\n{output}\n" + "="*60 + "\n")


❌ Limitations of `text2text-generation`

📖 Freeform Storytelling
📝 Prompt:
Continue the story: Once upon a time in a world of intelligent machines,
🤖 Output:
a robot was able to make a robot.

🧑‍🚀 Creative Writing
📝 Prompt:
Write a poem about space exploration.
🤖 Output:
i was a kid and i was a teenager and i was a kid and i was a kid and i was a kid and i was a kid and i was a kid and i was a kid and i was 



## 🏷️ `text-classification` Pipeline

## 🧠 What It Does:
This model doesn't generate text — instead, it **labels** text with a category.  
You give it something to read, and it tells you what “kind” of thing it is.

- **Purpose**: Predict a category or label from text
- **Pipeline**: `text-classification`
- **Best For**: Sentiment analysis, spam detection, topic labeling
- **Example Models**: `bert-base-uncased`, `distilbert-base-uncased`
- **Strengths**:
  - Fast and highly accurate for fixed-label tasks
  - Works out of the box for binary classification (e.g., positive/negative)
- **Limitations**:
  - Cannot generate output
  - Useless for translation, summarization, or reasoning

---

## ✅ Use Cases
- Sentiment analysis (positive/negative)
- Spam detection
- Topic classification
- Intent detection


In [12]:
# Load a simple classification model
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# Success cases
prompts = [
    ("I absolutely love this new AI assistant!", "😊 Positive Sentiment"),
    ("This is the worst update I’ve ever seen.", "😠 Negative Sentiment"),
]

print("✅ Successful Examples Using `text-classification`\n" + "="*60 + "\n")

for prompt, label in prompts:
    output = classifier(prompt)[0]
    print(f"{label}\n📝 Prompt:\n{prompt}\n🏷️ Output:\n{output}\n" + "="*60 + "\n")


✅ Successful Examples Using `text-classification`

😊 Positive Sentiment
📝 Prompt:
I absolutely love this new AI assistant!
🏷️ Output:
{'label': 'POSITIVE', 'score': 0.9998743534088135}

😠 Negative Sentiment
📝 Prompt:
This is the worst update I’ve ever seen.
🏷️ Output:
{'label': 'NEGATIVE', 'score': 0.9997844099998474}



In [13]:
# Fail cases: tasks that require actual output generation
fail_prompts = [
    ("Translate this sentence into Spanish: I love data science.", "🌍 Translation"),
    ("Summarize: Artificial intelligence is a field focused on building intelligent machines...", "🧾 Summarization")
]

print("❌ Limitations of `text-classification`\n" + "="*60 + "\n")

for prompt, label in fail_prompts:
    output = classifier(prompt)[0]
    print(f"{label}\n📝 Prompt:\n{prompt}\n🏷️ Output:\n{output}\n" + "="*60 + "\n")


❌ Limitations of `text-classification`

🌍 Translation
📝 Prompt:
Translate this sentence into Spanish: I love data science.
🏷️ Output:
{'label': 'POSITIVE', 'score': 0.9994205236434937}

🧾 Summarization
📝 Prompt:
Summarize: Artificial intelligence is a field focused on building intelligent machines...
🏷️ Output:
{'label': 'POSITIVE', 'score': 0.9885655045509338}





## ❓ `question-answering` Pipeline

## 🧠 What It Does:
Unlike chatbots or text generation, this pipeline **answers a question using a provided context**.  
It doesn’t guess — it **extracts** the answer from a given passage.

> 🧠 Think of it as:  
> “Read this paragraph and find the answer *inside it*.”

---

## ✅ Good For:
- Closed-book question answering
- FAQ bots with known documents
- Extractive QA in RAG systems (Retrieval-Augmented Generation)

### ❓ `question-answering`
- **Purpose**: Extract an answer from a provided context paragraph
- **Pipeline**: `question-answering`
- **Best For**: FAQs, document-based QA, extractive RAG systems
- **Example Models**: `bert-large-uncased-whole-word-masking-finetuned-squad`, `distilbert-base-uncased-distilled-squad`
- **Strengths**:
  - Great for “open-book” style QA
  - Fast and accurate when context is included
- **Limitations**:
  - Fails without a context
  - Cannot answer general or creative questions

In [15]:
# Load a pretrained QA model
qa = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Provide context and ask a question
examples = [
    (
        "Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize.",
        "Who was Marie Curie?",
        "🧬 Biography Fact"
    ),
    (
        "The capital of France is Paris. It is known for its culture, art, and landmarks like the Eiffel Tower.",
        "What is the capital of France?",
        "🗺️ Simple Fact Extraction"
    )
]

print("✅ Successful Examples Using `question-answering`\n" + "="*60 + "\n")

for context, question, label in examples:
    output = qa(question=question, context=context)
    print(f"{label}\n❓ Question: {question}\n📚 Context: {context}\n✅ Answer: {output['answer']}\n" + "="*60 + "\n")


✅ Successful Examples Using `question-answering`

🧬 Biography Fact
❓ Question: Who was Marie Curie?
📚 Context: Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize.
✅ Answer: a physicist and chemist

🗺️ Simple Fact Extraction
❓ Question: What is the capital of France?
📚 Context: The capital of France is Paris. It is known for its culture, art, and landmarks like the Eiffel Tower.
✅ Answer: Paris



In [23]:
fail_examples = [
    ("What is the meaning of life?", "", "🌌 Philosophical Question"),
    ("Who is the current president of the United States?", "", "📰 Real-time Fact"),
]

print("❌ Limitations of `question-answering`\n" + "="*60 + "\n")

for question, context, label in fail_examples:
    try:
        output = qa(question=question, context=context)
        print(f"{label}\n❓ Question: {question}\n📚 Context: [EMPTY]\n❌ Output: {output['answer']}\n" + "="*60 + "\n")
    except ValueError:
        print(f"{label}\n❓ Question: {question}\n📚 Context: [EMPTY]\n⚠️ Skipped: Question-answering requires context.\n" + "="*60 + "\n")


❌ Limitations of `question-answering`

🌌 Philosophical Question
❓ Question: What is the meaning of life?
📚 Context: [EMPTY]
⚠️ Skipped: Question-answering requires context.

📰 Real-time Fact
❓ Question: Who is the current president of the United States?
📚 Context: [EMPTY]
⚠️ Skipped: Question-answering requires context.



> ⚠️ The `question-answering` pipeline cannot work without a context paragraph.
> If you provide an empty string or leave it out, the model will raise an error:
> `ValueError: context cannot be empty`
>
> This reinforces that QA models are extractive — they don’t guess, they **retrieve** answers from known input.


## 🔬 Model Naming: `distilbert-base-uncased-…`

You're seeing:

- `distilbert-base-uncased-finetuned-sst-2-english` → used in **text-classification**
- `distilbert-base-uncased-distilled-squad` → used in **question-answering**

So what’s going on here?

---

## 🧠 First: What is DistilBERT?

**DistilBERT** is a smaller, faster version of **BERT**.  
It was created using a technique called **knowledge distillation**, which "compresses" a large model into a lighter one with minimal loss in accuracy.

| Feature        | BERT                      | DistilBERT                   |
|----------------|---------------------------|------------------------------|
| Size           | ~110M+ parameters         | ~66M parameters              |
| Speed          | Slower                    | 60% faster inference         |
| Accuracy       | Slightly higher           | Slightly reduced (~97%)      |
| Use Case       | Best when performance > speed | Best when speed matters   |

---

## 🔧 Second: Decoding the Model Names

### 1️⃣ `distilbert-base-uncased-finetuned-sst-2-english`

- `distilbert-base`: the **base DistilBERT model**
- `uncased`: lowercase-only vocabulary (doesn’t distinguish between `Apple` and `apple`)
- `finetuned-sst-2-english`: fine-tuned on the **SST-2 dataset** for **sentiment classification**

✅ Ideal for **text-classification**  
→ Returns "POSITIVE" or "NEGATIVE"

---

### 2️⃣ `distilbert-base-uncased-distilled-squad`

- Same base model: `distilbert-base-uncased`
- Fine-tuned on the **SQuAD dataset** (Stanford Question Answering Dataset)
- Task: Answer questions from a given paragraph

✅ Ideal for **question-answering**  
→ Returns answers extracted from context

---

## 🎯 TL;DR

| Model Name | Task | Dataset | Pipeline |
|------------|------|---------|----------|
| `distilbert-base-uncased-finetuned-sst-2-english` | Sentiment classification | SST-2 | `text-classification` |
| `distilbert-base-uncased-distilled-squad` | QA (extractive) | SQuAD | `question-answering` |

They share the same **core model architecture**, but are **fine-tuned for different tasks**.





## 🧾 `summarization` Pipeline

## 🧠 What It Does:
This pipeline takes a **long input** and returns a **shorter version** that preserves the main ideas.

> Think of it like an AI-powered TL;DR engine.

It’s great for:
- Articles
- Reports
- Emails
- Transcripts

---

### 🧾 `summarization`
- **Purpose**: Condense long text into a shorter summary
- **Pipeline**: `summarization`
- **Best For**: News articles, documentation, reports, lecture notes
- **Example Models**: `t5-small`, `facebook/bart-large-cnn`
- **Strengths**:
  - Extracts core ideas from longer input
  - Works well for compressing verbose content
- **Limitations**:
  - Can’t answer questions or follow instructions
  - May hallucinate or miss nuance in very short or vague prompts
---

## ✅ Good Models
- `facebook/bart-large-cnn`
- `t5-small`, `t5-base`

We’ll use `t5-small` for speed.



In [20]:
from transformers import pipeline

# Load a small summarization model
summarizer = pipeline("summarization", model="t5-small")

# Good examples
texts = [
    (
        "Artificial intelligence is a field of computer science focused on building systems that can perform tasks typically requiring human intelligence, such as understanding language, recognizing patterns, and making decisions.",
        "🤖 Summary of AI Definition"
    ),
    (
        "In 1969, the first humans landed on the Moon as part of NASA’s Apollo 11 mission. Neil Armstrong became the first person to walk on the Moon, followed by Buzz Aldrin. This historic event marked a major milestone in space exploration.",
        "🌕 Summary of Moon Landing"
    )
]

print("✅ Successful Examples Using `summarization`\n" + "="*60 + "\n")

for text, label in texts:
    output = summarizer(text, max_length=40, min_length=10, do_sample=False)[0]["summary_text"]
    print(f"{label}\n📚 Input:\n{text}\n🧾 Summary:\n{output}\n" + "="*60 + "\n")


✅ Successful Examples Using `summarization`

🤖 Summary of AI Definition
📚 Input:
Artificial intelligence is a field of computer science focused on building systems that can perform tasks typically requiring human intelligence, such as understanding language, recognizing patterns, and making decisions.
🧾 Summary:
artificial intelligence is a field of computer science focused on building systems that can perform tasks typically requiring human intelligence .

🌕 Summary of Moon Landing
📚 Input:
In 1969, the first humans landed on the Moon as part of NASA’s Apollo 11 mission. Neil Armstrong became the first person to walk on the Moon, followed by Buzz Aldrin. This historic event marked a major milestone in space exploration.
🧾 Summary:
in 1969, the first humans landed on the moon as part of NASA's Apollo 11 mission . the event marked a major milestone in space exploration .



In [21]:
# Fail cases — wrong tool for the job
bad_prompts = [
    ("What is the capital of Canada?", "📍 Direct Q&A"),
    ("Translate: I love space exploration.", "🌍 Translation Task")
]

print("❌ Limitations of `summarization`\n" + "="*60 + "\n")

for prompt, label in bad_prompts:
    output = summarizer(prompt, max_length=40, min_length=10, do_sample=False)[0]["summary_text"]
    print(f"{label}\n📝 Prompt:\n{prompt}\n🧾 Output:\n{output}\n" + "="*60 + "\n")


❌ Limitations of `summarization`

📍 Direct Q&A
📝 Prompt:
What is the capital of Canada?
🧾 Output:
what is the capital of Canada? if you live in a city in the u.s., it is a capital of the capital.

🌍 Translation Task
📝 Prompt:
Translate: I love space exploration.
🧾 Output:
translation: I love space exploration . a new translation of the translation .





## 💬 `conversational` Pipeline

### 🧠 What It Does:
This pipeline is designed for **chat-like interactions**, where the model can **remember previous messages** and continue a dialogue.

> It simulates a **conversation history** — you give it a list of messages and it replies accordingly.

---

## 🗨️ Good For:
- Chatbots
- Virtual assistants
- Multi-turn conversations
- Social dialog modeling (like talking to a character)

---

## 🧠 Powered by models like:
- `microsoft/DialoGPT-small`
- `facebook/blenderbot-400M-distill`

These models are **fine-tuned for dialogue**, not just text generation.



In [1]:
import transformers
print(transformers.__version__)

# !pip uninstall -y transformers
# !pip install transformers

4.51.3


Absolutely — let’s break it down clearly and concisely:

---

## 💬 `conversational` Pipeline (vs. `text-generation`)

### 🎯 Purpose
The `conversational` pipeline is built specifically to simulate **back-and-forth human conversations**, whereas `text-generation` is designed to **continue text from a prompt**.

---

## 🧠 Key Differences

| Feature                  | `conversational`                               | `text-generation`                        |
|--------------------------|-----------------------------------------------|------------------------------------------|
| **Goal**                 | Chat with memory                              | Generate open-ended text                 |
| **Input Format**         | Dialogue turns (via `Conversation` object)    | One continuous prompt                    |
| **Context Awareness**    | Maintains multi-turn memory                   | No built-in memory                       |
| **Good For**             | Chatbots, virtual assistants, characters      | Storytelling, brainstorming, free text   |
| **Typical Models**       | `DialoGPT`, `Blenderbot`                      | `GPT2`, `Falcon`, `LLaMA`                |
| **Response Style**       | Conversational, short, casual                 | Flowing, longform, creative              |
| **Trained On**           | Dialogue datasets like Reddit conversations   | Broad text (books, web, Wikipedia, etc.) |

---

### 🧪 Example Prompts

| Input                           | `conversational` Response         | `text-generation` Response                        |
|----------------------------------|------------------------------------|----------------------------------------------------|
| “Hi there!”                     | “Hey! How can I help?”            | “Hi there! I’m writing to let you know about...”   |
| “Tell me about AI agents.”     | “They’re used for automating tasks.” | “Tell me about AI agents. They are complex systems...” |
| “How’s your day going?”        | “Pretty good! How about you?”     | “How’s your day going? The weather outside is...”  |

---

### 🔍 Summary

- Use **`text-generation`** when you want **open-ended, creative writing**.
- Use **`conversational`** when you want **chat-style, multi-turn dialogue** with a more casual, interactive tone.



In [7]:
from transformers import pipeline, Conversation

# Load the conversational model
chatbot = pipeline("conversational", model="microsoft/DialoGPT-small")

# Create multi-turn conversations
print("✅ Successful Examples Using `conversational`\n" + "="*60 + "\n")

conv1 = Conversation("Hi there!")
chatbot(conv1)
print("🗨️ 1st message:", conv1.messages[-1]['content'])

conv1.add_user_input("How are you today?")
chatbot(conv1)
print("🗨️ 2nd message:", conv1.messages[-1]['content'])

conv1.add_user_input("What do you think about AI agents?")
chatbot(conv1)
print("🗨️ 3rd message:", conv1.messages[-1]['content'])

print("="*60 + "\n")

In [ ]:
fail_prompts = [
    "Translate this into Spanish: I love machine learning.",
    "Summarize: Artificial intelligence is a field that focuses on...",
    "What is the capital of France?"
]

print("❌ Limitations of `conversational`\n" + "="*60 + "\n")

for prompt in fail_prompts:
    conv = Conversation(prompt)
    chatbot(conv)
    print(f"📝 Prompt:\n{prompt}\n🤖 Output:\n{conv.messages[-1]['content']}\n" + "="*60 + "\n")


### clean widgets

In [9]:
import json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

notebook_path = "/content/drive/My Drive/AI AGENTS/005_Pipelines_Models_Comparison.ipynb"

# Load the notebook JSON
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = json.load(f)

# 1. Remove widgets from notebook-level metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]
    print("✅ Removed notebook-level 'widgets' metadata.")

# 2. Remove widgets from each cell's metadata
for i, cell in enumerate(nb.get("cells", [])):
    if "metadata" in cell and "widgets" in cell["metadata"]:
        del cell["metadata"]["widgets"]
        print(f"✅ Removed 'widgets' from cell {i}")

# Save the cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    json.dump(nb, f, indent=2)

print("✅ Notebook deeply cleaned. Try uploading to GitHub again.")

Mounted at /content/drive
✅ Notebook deeply cleaned. Try uploading to GitHub again.
